# Parallel TED subtitles dataset

Let's construct the TED subtitles dataset.

In [1]:
import re
import numpy as np
import pickle
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import codecs
import os,glob
import glob
import re

#natasha Russian word tokenizer
from razdel import tokenize

#Chienese word tokenizer
import jieba

In [243]:
jieba.lcut("但有個聲音不斷喚著我：「拜託，堅持下去。")

['但',
 '有',
 '個',
 '聲音',
 '不斷',
 '喚著',
 '我',
 '：',
 '「',
 '拜',
 '託',
 '，',
 '堅持',
 '下去',
 '。']

### At first let's try to get transriptions from the official site (not working).

In [82]:
def enlist_talk_names(path,dict_):
    with urllib.request.urlopen(path) as url:
        r = url.read()
        soup = BeautifulSoup(r)
        talks= soup.find_all("a",class_='')
        for i in talks:
            if i.attrs['href'].find('/talks/')==0 and dict_.get(i.attrs['href'])!=1:
                dict_[i.attrs['href']]=1

    return dict_

In [76]:
#all_talk_names={}
#for i in range(1,10000):
#    path='https://www.ted.com/talks?page=%d'%(i)
#    all_talk_names=enlist_talk_names(path,all_talk_names)

In [ ]:
def extract_talk(path,talk_name):
    r=urllib.urlopen(path).read()
    soup=BeautifulSoup(r)
    df=pd.DataFrame()
    print(path)
    for i in soup.findAll('link'):
        if i.get('href')!=None and i.attrs['href'].find('?language=')!=-1:
            lang=i.attrs['hreflang']
            path=i.attrs['href']
            r1=urllib.urlopen(path).read()
            soup1=BeautifulSoup(r1)
            time_frame=[]
            text_talk=[]
            for i in soup1.findAll('span',class_='talk-transcript__fragment'):
                time_frame.append(i.attrs['data-time'])
                text_talk.append(i.text.replace('\n',' '))
            #print len(time_frame),len(text_talk)
            df1=pd.DataFrame()
            df1[lang]=text_talk
            df1[lang+'_time_frame']=time_frame
            df=pd.concat([df,df1],axis=1)
    df.to_csv(talk_name+'.csv',sep='\t',encoding='utf-8')

In [80]:
for i in all_talk_names:
    extract_talk('https://www.ted.com'+i+'/transcript',i[7:])

TED is blocking multiple connections. Let's find another way.

### Search parallel corpuses on Github (not working)

Corpus at this link seems to be great. 

https://github.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/tree/master/Bilingual_Parallel_Corpus

In [50]:
corpus1 = pd.read_csv('D:\ml\zh-ru\parallel_corpus_1.txt', delimiter=':', names=['sent_id','lang_code','text'])
corpus2 = pd.read_csv('D:\ml\zh-ru\parallel_corpus_2.txt', delimiter=':', names=['sent_id','lang_code','text'])

In [56]:
corpus1[corpus1.sent_id == "132112"]

,sent_id,lang_code,text
1376490,132112,ar,"و هم الآن يصنعون خليطا,"
1376491,132112,nl,"Ze zijn nu bezig een cocktail,"
1376492,132112,fr,"Ils développent actuellement un cocktail,"
1376493,132112,de,"Sie machen gerade einen Cocktail,"
1376494,132112,he,"כיום הם מייצרים קוקטייל,"
1376495,132112,it,In questo momento si stanno preparando
1376496,132112,ru,Они сейчас готовят коктейль
1376497,132112,es,"Actualmente están haciendo un cóctel,"
1376498,132112,ar,"و هم الآن يصنعون خليطا,"
1376499,132112,pt-br,"Atualmente eles estão fazendo um coquetel,"


In [57]:
corpus2[corpus2.sent_id == "132112"]

,sent_id,lang_code,text
555789,132112,zh-cn,但是有一个声音一直在呼唤我：“来，别离开我。“
555790,132112,zh-tw,但有個聲音不斷喚著我：「拜託，堅持下去。」
555791,132112,ja,でも別の声が言います
555793,132112,ko,하지만 또다른 목소리가


In [53]:
corpus1.head()

,sent_id,lang_code,text
0,1,ar,بالنسبة لي، تبدأ هذه القصة قبل حوالي 15 عاماً ...
1,1,nl,Voor mij begint dit verhaal ongeveer 15 jaar g...
2,1,fr,"Pour moi, cette histoire a commencé il y a 15 ..."
3,1,de,Für mich beginnt diese Geschichte vor ungefähr...
4,1,he,"עבורי, הסיפור הזה התחיל בערך לפני 15 שנים,"


In [18]:
corpus2.head()

,sent_id,lang_code,text
0,1,zh-cn,对于我来说，这个故事是15年前开始的。
1,1,zh-tw,大約15年前
2,1,ja,話は15年前にさかのぼります
3,1,ko,"제게 있어서, 이 이야기는 15년 전에 시작했죠,"
4,2,zh-cn,当时我是芝加哥大学安养院的医生，


In [59]:
corpus1[corpus1.lang_code == 'ru'].shape

(345542, 3)

In [24]:
corpus2[corpus2.lang_code == 'zh-cn'].shape

(384109, 3)

In [39]:
zhru_df = pd.merge(corpus1[corpus1.lang_code == 'ru'].loc[:,['sent_id', 'text']], \
                   corpus2[corpus2.lang_code == 'zh-cn'].loc[:,['sent_id', 'text']], on='sent_id')

zhru_df.columns = ['sent_id', 'ru','zh']

In [60]:
zhru_df.head()

,sent_id,ru,zh
0,1,"Для меня всё началось лет 15 тому назад, когда...",对于我来说，这个故事是15年前开始的。
1,2,в хосписе при Чикагском Университете.,当时我是芝加哥大学安养院的医生，
2,3,Я оказывал помощь умирающим и их семьям,在芝加哥的南边地区
3,4,в районе Саут Сайд города Чикаго.,照顾临终的病人和他们的亲属。
4,5,"И я видел, что происходит на конечной стадии з...",我借此来观察疾病晚期


Unfortunately, half of the files are on the Git LFS storage and not available because author didn't upgrade the plan. And from existed files we can make parallel corpus, but it is not reliable (sentence ID's are not corresponding in a right way).

### Kaggle datasets

So, I've found the following dataset on Kaggle.

https://www.kaggle.com/goweiting/ted-talks-transcript

It's a large bunch of the subtitle files that we need to process to make the useful stuff.

In [6]:
import os
import glob
path = "D:\\ml\\ted-talks-transcript\\tedDirector\\subtitles"

In [11]:
id = 0
subtitles = {}

for d in [dirpath for (dirpath, dirnames, filenames) in os.walk(path)][1:]:
    types = ['\*.ru.vtt', '\*.zh-CN.vtt']
    pair = []
    
    for t in types:
        files = glob.glob(d + t)
        if len(files) == 1:
            pair.extend(files)
    
    if len(pair) == 2:
        subtitles[id] = pair
        id += 1

In [12]:
len(subtitles)

2309

In [5]:
subtitles[0]

['D:\\ml\\ted-talks-transcript\\tedDirector\\subtitles\\-0v7mTvJ8M4\\-0v7mTvJ8M4.ru.vtt',
 'D:\\ml\\ted-talks-transcript\\tedDirector\\subtitles\\-0v7mTvJ8M4\\-0v7mTvJ8M4.zh-CN.vtt']

In [6]:
[t.text for t in tokenize('диван-кровать стоит у стены')]

['диван-кровать', 'стоит', 'у', 'стены']

In [8]:
output_ru = 'D:\\ml\\ted-talks-transcript\\output_ru.txt'
output_zh = 'D:\\ml\\ted-talks-transcript\\output_zh.txt'
tokens_ru = 'D:\\ml\\ted-talks-transcript\\tokens_ru.txt'
tokens_zh = 'D:\\ml\\ted-talks-transcript\\tokens_zh.txt'

In [239]:
import random
rand = random.randint(0, 2309)

with open(output_ru, mode='w', encoding='utf-8') as out_ru, open(output_zh, mode='w', encoding='utf-8') as out_zh, \
        open(tokens_ru, mode='w', encoding='utf-8') as tok_ru, open(tokens_zh, mode='w', encoding='utf-8') as tok_zh:
    #or id in [id for id in subtitles][rand:rand+1]:  
    for id in [id for id in subtitles]:  
        strings = {}
        textlines = {'ru': [], 'zh':[]}

        sentences = {}
        textlines_sent = {'ru': [], 'zh':[]}
        
        for file in subtitles[id]:
            #print(file)
            timestamp = ''
            
            sentence_acc = ""
            
            with open(file, mode='r', encoding='utf-8') as f:                
                isRu = bool(re.search('ru\.vtt$', file))
                if isRu:
                    lang = 'ru'
                else:
                    lang = 'zh'
                
                for line in f.readlines():
                    isTimestamp = re.match("^\d\d:\d\d:\d\d\.\d\d\d.*", line)
                    
                    if isTimestamp:
                        if timestamp in strings:
                            strings[timestamp][lang] = ' '.join(textlines[lang])
                            textlines = {'ru': [], 'zh':[]}
                            #print('timestamp')

                        #comes out that the second date in timing is not reliable. taking the start time as key.
                        timestamp = line[:16]
                        if timestamp not in strings:
                            strings[timestamp] = {'ru': '', 'zh':''}
                    else: 
                        #filter beginning of the transcription
                        if not isTimestamp:
                            if (re.match("^Переводчик*", line) != None) | (re.match("^Редактор*", line) != None) | (re.match("^\(Смех\)*", line) != None) | (re.match("^\(Музыка\)*", line) != None) | (re.match("^\(Аплодисменты\)*", line) != None) | (re.match("^Спасибо*", line) != None):
                                textlines = {'ru': [], 'zh':[]}
                                continue
                        
                        if timestamp in strings:
                            #print('true', lang)
                            if len(line.strip()) > 0:
                                textlines[lang].append(line.replace('\t',' ').strip())
                            #print(textlines[lang])

        for ts in strings:
            if (len(strings[ts]['ru']) > 2) & (len(strings[ts]['zh']) > 2):
                str_ru = strings[ts]['ru']
                str_zh = strings[ts]['zh']
                
                out_ru.write(str_ru + '\n')
                out_zh.write(str_zh + '\n')
                
                #print(' '.join([t.text for t in tokenize(str_ru)]))
                #print(' '.join(jieba.lcut(str_zh)))
                
                tok_ru.write(' '.join([t.text for t in tokenize(str_ru)]) + '\n')
                tok_zh.write(' '.join(jieba.lcut(str_zh)) + '\n')
                

In [236]:
#707839
#552370

In [9]:
sent_ru = 'D:\\ml\\ted-talks-transcript\\sent_ru_ext.txt'
sent_zh = 'D:\\ml\\ted-talks-transcript\\sent_zh_ext.txt'

with open(output_ru, mode='r', encoding='utf-8') as out_ru, open(output_zh, mode='r', encoding='utf-8') as out_zh, \
    open(sent_ru, mode='w', encoding='utf-8') as sen_ru, open(sent_zh, mode='w', encoding='utf-8') as sen_zh:
    
    start = True
    end = False
    sentence_ru = ""
    sentence_zh = ""
    
    for line_ru in out_ru.readlines():
        line_zh = out_zh.readline()
        
        start = line_ru[:1].upper() == line_ru[:1]            
        end = re.match(".*[\.!?…]$", line_ru) != None
        
#         print(line_ru)
#         print(line_zh)
                    
        sentence_ru = sentence_ru + " " + line_ru.strip()
        sentence_zh = sentence_zh + " " + line_zh.strip()
            
        if end:  
            sent_len = len([t.text for t in tokenize(sentence_ru)])
            if (sent_len > 5) * (sent_len < 50):
                sen_ru.write(sentence_ru + '\n')
                sen_zh.write(sentence_zh + '\n')
                #print(sentence_ru)
                #print(sentence_zh)                
                #print()
                
            sentence_ru = ""
            sentence_zh = ""

        
        
    

#### Refinining

In [3]:
#df_corpus = pd.read_csv('D:\\ml\\ted-talks-transcript\\output.txt', delimiter='\t', names=['ru', 'zh'])

In [ ]:
with open(output_ru, mode='r', encoding='utf-8') as out_ru